## 数据处理

In [ ]:
# 切分数据 按照章节进行切分 其实并不通用
import re

def is_chapter(line):
    pattern = r'第.*章$'

i = 0
file_name = 'chapter' + str(i) + '.txt'

pre_text = ''

with open('./test.txt') as f:
    for line in f.readlines():
        if is_chapter(line[0:3]):
            i += 1
            file_name = 'chapter' + str(i) + '.txt'
            line = pre_text + '\n' +line

        with open('./chapter' + file_name,'a') as f_chapter:
            f_chapter.write(line)

## 构建本地数据库

In [4]:
# 设置矢量数据库
# 要创建本地 Milvus 矢量数据库，只需MilvusClient通过指定一个文件名来实例化一个来存储所有数据，例如“milvus_demo.db”。

from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# 生成了.milvus_demo.db.lock 和 milvus_demo.db


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# 创建收藏夹
# 在 Milvus 中，我们需要一个集合来存储向量及其相关元数据。你可以把它想象成传统 SQL 数据库中的表。
# 创建集合时，你可以定义 schema 和 index 参数来配置向量规范，例如维度、索引类型和距离度量。还有一些复杂的概念可以优化索引以提高向量搜索性能。
# 现在，我们只关注基础知识，并尽可能使用默认值。至少，你只需要设置集合名称和集合的向量字段的维度。

if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [6]:
import os
file_path = '../../data/chunk'
def create_docs_from_txt(txt_file):
    doc = []
    for txt_file in os.listdir(file_path):
        if txt_file.endswith('.txt'):  # 确保只处理 .txt 文件
            sentence = ''
            with open(os.path.join(file_path, txt_file), "r") as f:

                for line in f.readlines():
                    sentence += str(line[:-1]) # :-1 是为了去掉每行末尾的换行符
                    # if len(sentence) > 500:
                    #     # 如果chunk_size大于embedding模型的max_token数
                        
                    #     sentence += '。'
                        # break
            doc.append(sentence)

    return doc

documents = create_docs_from_txt(file_path)

for line in documents:
    print(len(line))

208
343
222
3373
1118
1157
108
767
359
121
457


## 加载embedding模型

In [1]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_id = "../../nlp_gte_sentence-embedding_chinese-large"
pipeline_se = pipeline(Tasks.sentence_embedding,
                       model=model_id,
                       sequence_length=512
                       ) # sequence_length 代表最大文本长度，默认值为128
# 当输入包含“soure_sentence”与“sentences_to_compare”时，会输出source_sentence中首个句子与sentences_to_compare中每个句子的向量表示，以及source_sentence中首个句子与sentences_to_compare中每个句子的相似度。
inputs = {
        "source_sentence": ["吃完海鲜可以喝牛奶吗?"],
        "sentences_to_compare": [
            "不可以，早晨喝牛奶不科学",
            "吃了海鲜后是不能再喝牛奶的，因为牛奶中含得有维生素C，如果海鲜喝牛奶一起服用会对人体造成一定的伤害",
            "吃海鲜是不能同时喝牛奶吃水果，这个至少间隔6小时以上才可以。",
            "吃海鲜是不可以吃柠檬的因为其中的维生素C会和海鲜中的矿物质形成砷"
        ]
    }

result = pipeline_se(input=inputs)
print (result)



/home/cdjzxy/anaconda3/envs/vllm310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'OfflineModeIsEnabled' from 'datasets.utils.file_utils' (/home/cdjzxy/anaconda3/envs/vllm310/lib/python3.10/site-packages/datasets/utils/file_utils.py)

In [1]:
from datasets.utils.file_utils import is_offline_mode

if is_offline_mode():
    print("Offline mode is enabled.")
else:
    print("Offline mode is not enabled.")

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# 当输入仅含有soure_sentence时，会输出source_sentence中每个句子的向量表示以及首个句子与其他句子的相似度。
inputs2 = {
        "source_sentence": [
            "不可以，早晨喝牛奶不科学",
            "吃了海鲜后是不能再喝牛奶的，因为牛奶中含得有维生素C，如果海鲜喝牛奶一起服用会对人体造成一定的伤害",
            "吃海鲜是不能同时喝牛奶吃水果，这个至少间隔6小时以上才可以。",
            "吃海鲜是不可以吃柠檬的因为其中的维生素C会和海鲜中的矿物质形成砷"
        ]
}
result = pipeline_se(input=inputs2)
print (result)

In [ ]:
! pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer
import torch

# Load the model, optionally in float16 precision for faster inference
model = SentenceTransformer("BAAI/bge-multilingual-gemma2", model_kwargs={"torch_dtype": torch.float16})

# Prepare a prompt given an instruction
instruction = 'Given a web search query, retrieve relevant passages that answer the query.'
prompt = f'<instruct>{instruction}\n<query>'
# Prepare queries and documents
queries = [
    '我想购买一台10万元的电脑，请问应该走什么流程',
    '采购流程',
]
# documents = [
#     "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]

# Compute the query and document embeddings
query_embeddings = model.encode(queries, prompt=prompt)
document_embeddings = model.encode(documents)

# Compute the cosine similarity between the query and document embeddings
similarities = model.similarity(query_embeddings, document_embeddings)
print(similarities)
# tensor([[ 0.5591,  0.0164],
#         [-0.0026,  0.4993]], dtype=torch.float16)


/home/cdjzxy/anaconda3/envs/vllm310/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name BAAI/bge-multilingual-gemma2. Creating a new one with mean pooling.


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like BAAI/bge-multilingual-gemma2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [24]:
# 准备数据

# 用向量表示文本
# 首先，安装模型库。此软件包包含 PyTorch 等基本 ML 工具。如果您的本地环境从未安装过 PyTorch，则软件包下载可能需要一些时间。

from pymilvus import model


embedding_fn = model.DefaultEmbeddingFunction() # 默认的embeding模型 效果待评估 之前用过bge-4
 

docs = create_docs_from_txt(file_path)
vectors = embedding_fn.encode_documents(docs)

# print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]
data
# print("Data has", len(data), "entities, each with fields: ", data[0].keys())
# print("Vector dim:", len(data[0]["vector"]))


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: token_type_ids for the following indices
 index: 1 Got: 564 Expected: 512
 Please fix either the inputs or the model.

In [2]:
data

[{'id': 0,
  'vector': array([-9.93218827e-03,  4.68230158e-02, -3.67523572e-02, -8.06826358e-02,
         -2.61729236e-02,  3.73494580e-03,  9.90153161e-03, -8.76287179e-02,
         -1.55368276e-02,  5.37705159e-02, -8.65140391e-02, -1.18773152e-02,
          4.30215167e-02, -2.44343163e-02,  1.66521825e-02,  2.23483155e-02,
         -3.18444510e-02, -4.02236303e-03,  2.29591487e-02, -4.86641908e-02,
         -3.80946075e-03, -1.59916598e-03,  2.32127215e-02, -3.35543301e-02,
         -3.71057595e-02,  1.86745284e-03,  1.36479287e-02, -2.56726715e-02,
         -2.49356191e-02,  3.94119391e-02,  1.63484918e-02,  4.86203447e-02,
          8.02349209e-02, -4.49607191e-02, -7.76418066e-02, -5.16100374e-02,
         -3.21353005e-02,  3.18941746e-02,  1.29663178e-02, -5.76224691e-02,
          8.74985634e-02,  4.73525795e-02,  2.03681622e-02,  4.44319332e-02,
         -3.21467702e-02, -1.22552220e-02, -1.86714994e-02, -1.22509875e-02,
         -5.41155517e-03, -3.72956077e-03, -1.92851697e

## 插入数据

In [6]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


In [8]:
# 语义搜索
# 现在我们可以通过将搜索查询文本表示为向量来进行语义搜索，并在 Milvus 上进行向量相似性搜索。

# 向量搜索
# Milvus 可以同时接受一个或多个向量搜索请求。query_vectors 变量的值是一个向量列表，其中每个向量都是一个浮点数数组。

query_vectors = embedding_fn.encode_queries(["象"])

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)


data: ["[{'id': 2, 'distance': 1.0, 'entity': {'text': '大象', 'subject': 'history'}}, {'id': 0, 'distance': 1.0, 'entity': {'text': '老虎', 'subject': 'history'}}]"] 


In [9]:
# 带元数据过滤的向量搜索
# 您还可以在考虑元数据的值（在 Milvus 中称为“标量”字段，因为标量指的是非向量数据）的同时进行向量搜索。
# 这可以通过指定某些条件的过滤表达式来完成。让我们subject在以下示例中看看如何使用字段进行搜索和过滤。


docs = [
    "Machine learning has been used for drug design.",
    "Computational synthesis with AI algorithms predicts molecular properties.",
    "DDR1 is involved in cancers and fibrosis.",
]
vectors = embedding_fn.encode_documents(docs)
data = [
    {"id": 3 + i, "vector": vectors[i], "text": docs[i], "subject": "biology"}
    for i in range(len(vectors))
]

client.insert(collection_name="demo_collection", data=data)

res = client.search(
    collection_name="demo_collection",
    data=embedding_fn.encode_queries(["tell me AI related information"]),
    filter="subject == 'biology'",
    limit=2,
    output_fields=["text", "subject"],
)

print(res)


data: ["[{'id': 4, 'distance': 0.2703056335449219, 'entity': {'text': 'Computational synthesis with AI algorithms predicts molecular properties.', 'subject': 'biology'}}, {'id': 3, 'distance': 0.1642589271068573, 'entity': {'text': 'Machine learning has been used for drug design.', 'subject': 'biology'}}]"] 


## 读取数据

In [1]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")


In [3]:
res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)
res

data: ["{'id': 0, 'text': '老虎', 'subject': 'history'}", "{'id': 1, 'text': 'A狮子', 'subject': 'history'}", "{'id': 2, 'text': '大象', 'subject': 'history'}"] 

In [6]:
res = client.query(
    collection_name="demo_collection",
    ids=[0, 2],
    output_fields=["vector", "text", "subject"],
)
# print(res)